# Julia on EMR

First, let's install the [Elly](https://github.com/JuliaParallel/Elly.jl) package.

In [1]:
import Pkg; Pkg.add("Elly")

Then, let's spin up a Yarn Cluster Manager in "unmanaged" mode - that means our notebook has to stay running in order for the job to run to completion.

In addition, we need the IP address of the local Yarn host. We'll fetch that using `readchomp()`.

In [2]:
yarnhost = readchomp(`hostname -i`)

"10.0.44.26"

In [3]:
using Elly

ENV["HADOOP_USER_NAME"] = "hadoop"

yarncm = YarnManager(
    yarnhost=yarnhost,
    rmport=8032,
    schedport=8030,
    launch_timeout=60,
    unmanaged=true     # pass true when running in unmanaged mode
);

After that, we'll use the Distributed package to spin up 8 processors.

For this demo, we'll just have each processor report back it's ID.

In [4]:
using Distributed

env = Dict(
    "JULIA_LOAD_PATH"=>join([Base.LOAD_PATH..., "/usr/local/julia/packages"], ':'),
    "JULIA_DEPOT_PATH"=>join([Base.DEPOT_PATH..., "/usr/local/julia"], ':')
);
addprocs(yarncm; np=8, env=env);

@everywhere using Distributed
@everywhere println(readchomp(`hostname -i`))

10.0.44.26
      From worker 5:	10.0.44.26
      From worker 3:	10.0.44.26
      From worker 9:	10.0.44.26
      From worker 8:	10.0.44.26
      From worker 2:	10.0.44.26
      From worker 7:	10.0.44.26
      From worker 4:	10.0.44.26
      From worker 6:	10.0.44.26


And, like a good client, we always shut down our connections. :)

In [5]:
Elly.disconnect(yarncm);

      From worker 6:	
      From worker 6:	signal (15): Terminated
      From worker 6:	in expression starting at none:1
      From worker 6:	epoll_pwait at /lib64/libc.so.6 (unknown line)
      From worker 6:	uv__io_poll at /workspace/srcdir/libuv/src/unix/linux-core.c:309
      From worker 6:	uv_run at /workspace/srcdir/libuv/src/unix/core.c:379
      From worker 6:	jl_task_get_next at /buildworker/worker/package_linux64/build/src/partr.c:472
      From worker 6:	poptask at ./task.jl:760
      From worker 6:	wait at ./task.jl:768 [inlined]
      From worker 6:	task_done_hook at ./task.jl:494
      From worker 6:	_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2237 [inlined]
      From worker 6:	jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2419
      From worker 6:	jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1703 [inlined]
      From worker 6:	jl_finish_task at /buildworker/worker/package_linux64/build/src/task.c:208
   

Worker 5 terminated.
Worker 3 terminated.
Worker 2 terminated.
Worker 8 terminated.
Worker 6 terminated.
Worker 4 terminated.
Worker 7 terminated.
Worker 9 terminated.
